# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [31]:
# Import pandas
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%matplotlib inline

In [8]:
df = pd.read_csv('homepage_actions.csv')
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [11]:
df.action.value_counts()
df.id.value_counts()
df.group.value_counts()

control       4264
experiment    3924
Name: group, dtype: int64

In [24]:
# Control group total who viewed
df_ctrl_vw = df.loc[(df.group == 'control') & (df.action == 'view')]

df_ctrl_vw.head()

df_ctrl_vw.id.value_counts()

# Experimental group total who viewed
df_exp_vw = df.loc[(df.group == 'experiment') & (df.action == 'view')]

df_exp_vw.head()

df_exp_vw.id.value_counts()

878591    1
365888    1
445775    1
759651    1
335563    1
         ..
547480    1
437791    1
381331    1
599042    1
735865    1
Name: id, Length: 2996, dtype: int64

In [25]:
# Control group total who clicked
df_ctrl_cl = df.loc[(df.group == 'control') & (df.action == 'click')]

df_ctrl_cl.id.value_counts()

# Experimental group total who clicked
df_exp_cl = df.loc[(df.group == 'experiment') & (df.action == 'click')]

df_exp_cl.id.value_counts()

# Percentage of control & experimental group who clicked:
ctrl_cl_pc = round(len(df_ctrl_cl) / len(df_ctrl_vw), 2)
exp_cl_pc = round(len(df_exp_cl) / len(df_exp_vw), 2)

print(f'The control group percentage who also clicked is: {ctrl_cl_pc}')
print(f'The experimental group percentage who also clicked is: {exp_cl_pc}')


The control group percentage who also clicked is: 0.28
The experimental group percentage who also clicked is: 0.31


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

#### Question: is there a statistically significant difference between the percentage of clicks in the control group, and the percentage of clicks in the experimental group?

##### Ho: There is no difference between the mean percentage of clicks between viewers in the control and experiment groups.
##### Ha: The mean percentage of clicks in the experimental group will be greater than in the control group.

In [50]:
# Control group - create binary variables for 'click'
df_control = df.loc[df.group == 'control']

# Create binary columns for views and clicks

df_control['view_bin'] = None
df_control['click_bin'] = None

for i, row in df_control.iterrows():
    if row['action'] == 'view':
        df_control.loc[i, 'view_bin'] = 1
        df_control.loc[i, 'click_bin'] = 0
    elif row['action'] == 'click':
        df_control.loc[i, 'view_bin'] = 1
        df_control.loc[i, 'click_bin'] = 1

df_control



C:\Users\dougl\Anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\dougl\Anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,timestamp,id,group,action,view_bin,click_bin
3,2016-09-24 19:59:02.646620,671993,control,view,1,0
9,2016-09-24 21:00:12.278374,560027,control,view,1,0
23,2016-09-25 00:25:14.141290,281985,control,view,1,0
24,2016-09-25 01:14:48.603202,407864,control,view,1,0
25,2016-09-25 02:16:11.046654,342984,control,view,1,0
...,...,...,...,...,...,...
8178,2017-01-18 08:17:12.675797,616692,control,view,1,0
8181,2017-01-18 09:07:37.661143,795585,control,view,1,0
8182,2017-01-18 09:09:17.363917,795585,control,click,1,1
8186,2017-01-18 10:08:51.588469,505451,control,view,1,0


In [51]:
# Control data with unique id values only (duplicates dropped)
df_control_uniq = df_control.drop_duplicates(subset='id', keep='last')

df_control_uniq

,timestamp,id,group,action,view_bin,click_bin
3,2016-09-24 19:59:02.646620,671993,control,view,1,0
9,2016-09-24 21:00:12.278374,560027,control,view,1,0
23,2016-09-25 00:25:14.141290,281985,control,view,1,0
24,2016-09-25 01:14:48.603202,407864,control,view,1,0
25,2016-09-25 02:16:11.046654,342984,control,view,1,0
...,...,...,...,...,...,...
8177,2017-01-18 07:15:05.620801,839581,control,view,1,0
8178,2017-01-18 08:17:12.675797,616692,control,view,1,0
8182,2017-01-18 09:09:17.363917,795585,control,click,1,1
8186,2017-01-18 10:08:51.588469,505451,control,view,1,0


In [52]:
# Experimental group - create binary variables for 'click'
df_exp = df.loc[df.group == 'experiment']

# Create binary columns for views and clicks

df_exp['view_bin'] = None
df_exp['click_bin'] = None

for i, row in df_exp.iterrows():
    if row['action'] == 'view':
        df_exp.loc[i, 'view_bin'] = 1
        df_exp.loc[i, 'click_bin'] = 0
    elif row['action'] == 'click':
        df_exp.loc[i, 'view_bin'] = 1
        df_exp.loc[i, 'click_bin'] = 1

# Experimental data with unique id values only (duplicates dropped)
df_exp_uniq = df_exp.drop_duplicates(subset='id', keep='last')       

df_exp_uniq

C:\Users\dougl\Anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\dougl\Anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,timestamp,id,group,action,view_bin,click_bin
0,2016-09-24 17:42:27.839496,804196,experiment,view,1,0
1,2016-09-24 19:19:03.542569,434745,experiment,view,1,0
2,2016-09-24 19:36:00.944135,507599,experiment,view,1,0
4,2016-09-24 20:26:14.466886,536734,experiment,view,1,0
5,2016-09-24 20:32:25.712659,681598,experiment,view,1,0
...,...,...,...,...,...,...
8176,2017-01-18 07:07:50.090346,540466,experiment,view,1,0
8180,2017-01-18 08:54:56.879682,615849,experiment,click,1,1
8183,2017-01-18 09:11:41.984113,192060,experiment,view,1,0
8184,2017-01-18 09:42:12.844575,755912,experiment,view,1,0


In [54]:
# Import flatiron stats functions
import scipy.stats as stats

In [58]:
# Running a two-sample, onse-sided welch's t-test, since we cannot assume equal variance, 
# and we are looking to detect a difference.

t_stat, p_val = stats.ttest_ind(df_control_uniq.click_bin, df_exp_uniq.click_bin, equal_var=False)

if (p_val/2) >= 0.05:
    print(f'The p-value is {round(p_val/2, 3)}. The result is NOT significant at alpha = 0.05')
else:
    print(f'The p-value is {round(p_val/2, 3)}. The result IS significant at alpha = 0.05')

The p-value is 0.004. The result IS significant at alpha = 0.05


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [60]:
# expected_clicks = total_exp_views * ctrl_click_rate

expec_exper_clicks = 2996 * 0.28
print(round(expec_exper_clicks, 0))

839.0


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [70]:
sd = np.sqrt(2996 * 0.28 * (1-0.28))
print(sd)

24.576281248390693


In [71]:
act_clicks = df_exp.click_bin.sum()
est_clicks = expec_exper_clicks = 2996 * 0.28

z_score = (act_clicks - est_clicks) / sd

print(z_score)

3.6262605843117806


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [72]:
#Your code here
p_value = 1 - stats.norm.cdf(z_score)

print(p_value)

0.00014377758963901588


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Not exactly the same, but they both indicate a signficant result. Therefore, in this case we reject the null hypothesis that the experiment group mean clicks is equal or less than control group.**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.